In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import os



In [22]:
import pandas as pd

# Dictionnaire des timeframes
timeframes = {
    "1h": "database\data_with_indicators\\1h\BTC-USDT-USDT.csv",
    "2h": "database\data_with_indicators\\2h\BTC-USDT-USDT.csv",
    "4h": "database\data_with_indicators\\4h\BTC-USDT-USDT.csv"
}


<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Elodie\AppData\Local\Temp\ipykernel_18404\2658866293.py:5: SyntaxWarning: invalid escape sequence '\d'
  "1h": "database\data_with_indicators\\1h\BTC-USDT-USDT.csv",
C:\Users\Elodie\AppData\Local\Temp\ipykernel_18404\2658866293.py:6: SyntaxWarning: invalid escape sequence '\d'
  "2h": "database\data_with_indicators\\2h\BTC-USDT-USDT.csv",
C:\Users\Elodie\AppData\Local\Temp\ipykernel_18404\2658866293.py:7: SyntaxWarning: invalid escape sequence '\d'
  "4h": "database\data_with_indicators\\4h\BTC-USDT-USDT.csv"


In [23]:
def preprocess(df, features, label='Label_CT', time_steps=60):
    df = df.dropna()
    X = df[features]
    y = df[label]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    X_seq, y_seq = [], []
    for i in range(time_steps, len(X_scaled)):
        X_seq.append(X_scaled[i-time_steps:i])
        y_seq.append(y.iloc[i])
    return np.array(X_seq), np.array(y_seq)


In [24]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(32))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
def creer_label_variation(df, prix_col='close', seuil_pct=0.05):
    """
    Crée une colonne Label_CT selon la variation en % du prix (close) entre j et j-1.
    
    Args:
        df (pd.DataFrame): dataframe avec colonne prix
        prix_col (str): nom de la colonne prix (ex: 'close')
        seuil_pct (float): seuil en % (ex: 0.05 pour 0.05%)
        
    Returns:
        pd.Series: labels -1, 0, 1
    """
    variation_pct = df[prix_col].pct_change() * 100 
    labels = variation_pct.apply(lambda x: 1 if x > seuil_pct else (-1 if x < -seuil_pct else 0))
    labels.iloc[0] = 0  # premier label non défini, on met stable par défaut
    return labels


In [28]:
features = ['open','high','low','close','volume','rsi','rsima','macd','macd_diff','macd_signal',
            'stoch_%K','stoch_%D','lower_band','higher_band','ma_band','sma9','sma21',
            'Avg_day','Avg_corps','Avg_meches']

results = {}

# Créer un dossier pour stocker les modèles si non existant
os.makedirs("models", exist_ok=True)

for tf, path in timeframes.items():
    print(f"\n⏱️ Traitement timeframe : {tf}")
    
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'], unit='ms', errors='coerce')
    df = df.dropna(subset=['date'])

    X, y = preprocess(df, features, label='Label_CT', time_steps=60)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    model = build_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

    loss, acc = model.evaluate(X_test, y_test)
    print(f"✅ {tf} - Test Accuracy: {acc:.4f}")

    results[tf] = {
        'model': model,
        'history': history,
        'accuracy': acc
    }

    # Sauvegarde du modèle pour ce timeframe
    model_path = f"models/lstm_btc_10_{tf}.h5"
    model.save(model_path)
    print(f"💾 Modèle sauvegardé : {model_path}")



⏱️ Traitement timeframe : 1h


c:\Users\Elodie\Code\Ynov\MoneyTime\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


318/318 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9305 - loss: 0.2223
✅ 1h - Test Accuracy: 0.9432


💾 Modèle sauvegardé : models/lstm_btc_10_1h.h5

⏱️ Traitement timeframe : 2h


c:\Users\Elodie\Code\Ynov\MoneyTime\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9396 - loss: 0.2074


✅ 2h - Test Accuracy: 0.9455
💾 Modèle sauvegardé : models/lstm_btc_10_2h.h5

⏱️ Traitement timeframe : 4h


c:\Users\Elodie\Code\Ynov\MoneyTime\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9441 - loss: 0.1636


✅ 4h - Test Accuracy: 0.9505
💾 Modèle sauvegardé : models/lstm_btc_10_4h.h5
